In [1]:
import pandas as pd
import numpy as np

meta = pd.read_csv('meta_elevation.csv')

Now i will use the data available at [copernicus (human settlement data)](https://human-settlement.emergency.copernicus.eu/download.php?ds=bu)

I downloaded the data from 4 rectangles for poland which include population, build up surface and build up volume data for 2015 with 100 m resolution

In [ ]:
import rasterio
from rasterio.merge import merge
import os

def merge_rasters(file_list):
    src_files = []
    try:
        for fp in file_list:
            if not os.path.exists(fp):
                print(f"[WARNING] Plik nie istnieje: {fp} - pomijam.")
                continue
            src = rasterio.open(fp)
            src_files.append(src)

        if len(src_files) == 0:
            print("[ERROR] Brak poprawnych plików do scalania.")
            return None, None, None

        # Sprawdzenie czy wszystkie CRS są takie same
        crs_set = set([src.crs.to_string() for src in src_files])
        if len(crs_set) > 1:
            print("[ERROR] Różne układy współrzędnych CRS w plikach:")
            for src in src_files:
                print(f"  {src.name}: {src.crs}")
            # Można tu dodać rzutowanie, ale na razie kończymy
            return None, None, None

        # Scalanie
        mosaic, out_trans = merge(src_files)
        crs = src_files[0].crs

    except Exception as e:
        print(f"[ERROR] Wystąpił błąd podczas scalania: {e}")
        return None, None, None

    finally:
        for src in src_files:
            src.close()

    print("[INFO] Scalanie rasterów zakończone pomyślnie.")
    return mosaic, out_trans, crs


raster_files = ["1v.tif", "2v.tif", "3v.tif", "4v.tif"]
mosaic, transform, crs = merge_rasters(raster_files)
if mosaic is not None:
    print("Kształt scalonego rastra:", mosaic.shape)
    print("Transformacja:", transform)
    print("CRS:", crs)
else:
    print("Scalanie nie powiodło się.")

import rasterio

out_meta = {
    'driver': 'GTiff',
    'height': mosaic.shape[1],
    'width': mosaic.shape[2],
    'count': mosaic.shape[0],      # liczba kanałów/bandów
    'dtype': mosaic.dtype,
    'crs': crs,
    'transform': transform
}

output_path = 'volume.tif'

with rasterio.open(output_path, 'w', **out_meta) as dest:
    dest.write(mosaic)

print(f"Zapisano scalony raster do {output_path}")


[INFO] Scalanie rasterów zakończone pomyślnie.
Kształt scalonego rastra: (1, 20000, 20000)
Transformacja: | 100.00, 0.00,-41000.00|
| 0.00,-100.00, 7000000.00|
| 0.00, 0.00, 1.00|
CRS: PROJCS["World_Mollweide",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["central_meridian",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["Meter",1],AXIS["Easting",EAST],AXIS["Northing",NORTH]]
Zapisano scalony raster do volume.tif


In [ ]:
raster_files = ["1p.tif", "2p.tif", "3p.tif", "4p.tif"]
mosaic, transform, crs = merge_rasters(raster_files)
if mosaic is not None:
    print("Kształt scalonego rastra:", mosaic.shape)
    print("Transformacja:", transform)
    print("CRS:", crs)
else:
    print("Scalanie nie powiodło się.")

import rasterio

out_meta = {
    'driver': 'GTiff',
    'height': mosaic.shape[1],
    'width': mosaic.shape[2],
    'count': mosaic.shape[0],      # liczba kanałów/bandów
    'dtype': mosaic.dtype,
    'crs': crs,
    'transform': transform
}

output_path = 'population.tif'

with rasterio.open(output_path, 'w', **out_meta) as dest:
    dest.write(mosaic)

print(f"Zapisano scalony raster do {output_path}")

In [ ]:
raster_files = ["1s.tif", "2s.tif", "3s.tif", "4s.tif"]
mosaic, transform, crs = merge_rasters(raster_files)
if mosaic is not None:
    print("Kształt scalonego rastra:", mosaic.shape)
    print("Transformacja:", transform)
    print("CRS:", crs)
else:
    print("Scalanie nie powiodło się.")

import rasterio

out_meta = {
    'driver': 'GTiff',
    'height': mosaic.shape[1],
    'width': mosaic.shape[2],
    'count': mosaic.shape[0],
    'dtype': mosaic.dtype,
    'crs': crs,
    'transform': transform
}

output_path = 'surface.tif'

with rasterio.open(output_path, 'w', **out_meta) as dest:
    dest.write(mosaic)

print(f"Zapisano scalony raster do {output_path}")

In [25]:
import geopandas as gpd
from shapely.geometry import Point
import rasterio
from rasterio.mask import mask
import numpy as np


def raster_data(lon, lat, radius_m = 5000, mosaic_path='population.tif'):
    # 1. Punkt i bufor w EPSG:2180 (metryczny układ)
    gdf_point = gpd.GeoDataFrame({'geometry': [Point(lon, lat)]}, crs="EPSG:4326")
    gdf_point = gdf_point.to_crs(epsg=2180)
    buffer_geom = gdf_point.geometry.buffer(radius_m)

    # Otwieramy scalony raster
    with rasterio.open(mosaic_path) as src:
        # Rzutujemy bufor do CRS rastra
        buffer_proj = buffer_geom.to_crs(src.crs)

        # Maskujemy raster po buforze i wycinamy
        out_image, out_transform = mask(src, [buffer_proj.iloc[0]], crop=True)
        data = out_image[0].astype(float)

        nodata = src.nodata
        if nodata is not None:
            data[data == nodata] = np.nan

    # Liczymy statystyki
    suma = np.nansum(data)
    srednia = np.nanmean(data)
    mediana = np.nanmedian(data)
    maksimum = np.nanmax(data)
    minimum = np.nanmin(data)
    '''
    print(f"\nSTATYSTYKI W PROMIENIU {radius_m} m od punktu {lon},{lat}:")
    print(f"  Suma: {suma:.2f}")
    print(f"  Średnia: {srednia:.2f}")
    print(f"  Mediana: {mediana:.2f}")
    print(f"  Maksimum: {maksimum:.2f}")
    print(f"  Minimum: {minimum:.2f}")
    '''
    return suma



In [26]:
population = []
surface = []
volume = []

for stacja, lon, lat in zip(meta.index, meta['lon'], meta['lat']):
    print(f"Pobieram dane dla {stacja} ({lat}, {lon})...")
    sumapop = raster_data(lon, lat, 5000, 'population.tif')
    sumavol = raster_data(lon, lat, 5000, 'volume.tif')
    sumasur = raster_data(lon, lat, 5000, 'surface.tif')
    population.append(sumapop)
    volume.append(sumavol)
    surface.append(sumasur)

Pobieram dane dla 0 (51.204503, 16.180513)...
Pobieram dane dla 1 (51.31763, 15.431719)...
Pobieram dane dla 2 (50.768729, 16.269677)...
Pobieram dane dla 3 (51.103456, 17.059225)...
Pobieram dane dla 4 (51.150391, 15.008175)...
Pobieram dane dla 5 (53.151452, 18.132062)...
Pobieram dane dla 6 (53.491831, 18.752503)...
Pobieram dane dla 7 (53.028647, 18.666103)...
Pobieram dane dla 8 (53.662117, 17.934017)...
Pobieram dane dla 9 (52.029194, 23.149389)...
Pobieram dane dla 10 (51.13095, 23.514603)...
Pobieram dane dla 11 (51.273078, 22.551675)...
Pobieram dane dla 12 (50.716628, 23.290247)...
Pobieram dane dla 13 (51.75805, 19.529786)...
Pobieram dane dla 14 (51.776417, 19.452936)...
Pobieram dane dla 15 (51.404406, 19.696956)...
Pobieram dane dla 16 (52.747044, 15.246294)...
Pobieram dane dla 17 (51.642656, 15.127808)...
Pobieram dane dla 18 (51.939783, 15.518861)...
Pobieram dane dla 19 (49.969017, 20.439511)...
Pobieram dane dla 20 (50.010575, 19.949189)...
Pobieram dane dla 21 (49.6

In [32]:
meta['population'] = population
meta['built_up_surface'] = surface
meta['built_up_volume'] = volume
meta.to_csv('metaall.csv')
meta

,stacja,lat,lon,nearest_road_dist_m,total_road_length_km,building_count,elevation,population,built_up_surface,built_up_volume
0,DsLegAlRzecz,51.204503,16.180513,98.613062,942.949585,15745.0,124.0,101762.211892,6385774.0,45643207.0
1,DsOsieczow21,51.317630,15.431719,40.258760,148.954010,2334.0,179.0,2240.586464,343715.0,859436.0
2,DsWalbrzWyso,50.768729,16.269677,170.100903,817.813995,15897.0,441.0,102796.665402,4756994.0,22788216.0
3,DsWrocNaGrob,51.103456,17.059225,97.176639,1518.020860,60249.0,123.0,368776.490393,14934688.0,171569591.0
4,DsZgorBohGet,51.150391,15.008175,138.987771,988.524194,28363.0,213.0,83586.284188,6462650.0,34522676.0
...,...,...,...,...,...,...,...,...,...,...
56,WpPleszAlMic,51.884922,17.791106,86.378906,518.121034,9820.0,119.0,24809.861029,3404141.0,17342688.0
57,ZpKoszSpasow,54.207151,16.193236,27.102665,1019.729752,15411.0,48.0,108862.314139,5986037.0,39885827.0
58,ZpMyslZaBram,52.926283,14.862809,15.499193,290.059223,4069.0,65.0,13388.313425,1126673.0,4347116.0
59,ZpSzczAndrze,53.380975,14.663347,26.632511,653.938328,12399.0,14.0,58834.908799,4068844.0,23983848.0


In [33]:
data = pd.read_csv('stacje_pogoda_2016_2019.csv')
data

,Unnamed: 0,date,stacja,pm25,temperature_2m_min,wind_speed_10m_max,wind_direction_10m_dominant,cloud_cover_mean,surface_pressure_mean,dew_point_2m_mean,relative_humidity_2m_min,wind_speed_10m_mean,boundary_layer_height
0,0,2016-01-01 00:00:00+00:00,DsLegAlRzecz,59.30,-1.932500,11.753877,289.29013,100.000000,1010.14970,-1.338750,64.538155,7.212135,191.66667
1,1,2016-01-02 00:00:00+00:00,DsLegAlRzecz,54.70,-9.532500,22.896110,102.65439,59.958332,1009.82965,-10.611667,60.426315,18.588041,433.12500
2,2,2016-01-03 00:00:00+00:00,DsLegAlRzecz,69.00,-12.282500,18.709612,107.50388,49.416668,1003.64520,-15.309583,60.893390,17.861810,398.33334
3,3,2016-01-04 00:00:00+00:00,DsLegAlRzecz,80.10,-10.632501,20.696087,118.76107,92.458336,985.55080,-12.167916,55.315773,16.140474,477.91666
4,4,2016-01-05 00:00:00+00:00,DsLegAlRzecz,100.50,-7.082500,13.324863,86.64323,95.750000,985.00410,-8.424168,69.855020,10.575133,268.75000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
66912,66912,2018-12-28 00:00:00+00:00,ZpSzczec1Maj,4.90,4.139000,24.535526,298.53427,100.000000,1003.10750,4.574417,93.932650,19.015276,728.54170
66913,66913,2018-12-29 00:00:00+00:00,ZpSzczec1Maj,8.16,3.889000,17.555307,256.10280,99.958336,1004.09380,4.422333,93.853710,14.292187,552.08330
66914,66914,2018-12-30 00:00:00+00:00,ZpSzczec1Maj,4.35,1.689000,27.002400,298.98737,95.958336,1000.37670,3.041083,85.138250,19.555052,665.83330
66915,66915,2018-12-31 00:00:00+00:00,ZpSzczec1Maj,7.44,1.839000,20.883102,271.19727,98.791664,1009.56620,1.770250,82.750200,12.496941,395.62500


In [ ]:
data = pd.read_csv('stations_weather_2016_2019_1h.csv')
meta = pd.read_csv('metaall.csv')
all = pd.merge(data, meta, on=["stacja"], how="inner")
all
all.to_csv('all_data_1h.csv')
all

,Unnamed: 0_x,date,stacja,pm25,temperature_2m,relative_humidity_2m,wind_speed_10m,wind_direction_10m,cloud_cover,surface_pressure,...,Unnamed: 0_y,lat,lon,nearest_road_dist_m,total_road_length_km,building_count,elevation,population,built_up_surface,built_up_volume
0,0,2016-01-01 00:00:00+00:00,KpToruDziewu,NaN,1.8890,96.140360,5.154416,192.09474,100.0,1020.17566,...,7,53.028647,18.666103,97.101206,1339.475703,21483.0,66.0,139210.225255,9214588.0,83715434.0
1,1,2016-01-01 01:00:00+00:00,KpToruDziewu,NaN,1.6890,96.827040,5.959060,205.01686,95.0,1019.97314,...,7,53.028647,18.666103,97.101206,1339.475703,21483.0,66.0,139210.225255,9214588.0,83715434.0
2,2,2016-01-01 02:00:00+00:00,KpToruDziewu,NaN,1.2890,97.516304,6.193674,215.53777,94.0,1020.26465,...,7,53.028647,18.666103,97.101206,1339.475703,21483.0,66.0,139210.225255,9214588.0,83715434.0
3,3,2016-01-01 03:00:00+00:00,KpToruDziewu,NaN,1.0890,97.512300,7.421590,230.90614,98.0,1020.36066,...,7,53.028647,18.666103,97.101206,1339.475703,21483.0,66.0,139210.225255,9214588.0,83715434.0
4,4,2016-01-01 04:00:00+00:00,KpToruDziewu,NaN,1.0390,97.511280,8.049845,243.43501,100.0,1020.35970,...,7,53.028647,18.666103,97.101206,1339.475703,21483.0,66.0,139210.225255,9214588.0,83715434.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350635,350635,2019-12-31 19:00:00+00:00,ZpSzczAndrze,9.41298,3.6585,91.862840,14.658676,294.67685,17.0,1027.27320,...,59,53.380975,14.663347,26.632511,653.938328,12399.0,14.0,58834.908799,4068844.0,23983848.0
350636,350636,2019-12-31 20:00:00+00:00,ZpSzczAndrze,34.82470,3.1585,93.482430,12.904882,292.98874,9.0,1027.46350,...,59,53.380975,14.663347,26.632511,653.938328,12399.0,14.0,58834.908799,4068844.0,23983848.0
350637,350637,2019-12-31 21:00:00+00:00,ZpSzczAndrze,30.00260,2.7585,94.468340,13.104197,290.92456,10.0,1027.95410,...,59,53.380975,14.663347,26.632511,653.938328,12399.0,14.0,58834.908799,4068844.0,23983848.0
350638,350638,2019-12-31 22:00:00+00:00,ZpSzczAndrze,27.14000,2.2585,95.466354,11.966954,285.70870,4.0,1028.14450,...,59,53.380975,14.663347,26.632511,653.938328,12399.0,14.0,58834.908799,4068844.0,23983848.0


In [3]:
data = pd.read_csv('stations_weather_2016_2019.csv')
meta = pd.read_csv('metaall.csv')
all = pd.merge(data, meta, on=["stacja"], how="inner")
all
all.to_csv('all_data.csv')
all

,Unnamed: 0_x,date,stacja,pm25,temperature_2m_min,wind_speed_10m_max,wind_direction_10m_dominant,cloud_cover_mean,surface_pressure_mean,dew_point_2m_mean,...,Unnamed: 0_y,lat,lon,nearest_road_dist_m,total_road_length_km,building_count,elevation,population,built_up_surface,built_up_volume
0,0,2016-01-01 00:00:00+00:00,DsLegAlRzecz,59.30,-1.932500,11.753877,289.29013,100.000000,1010.14970,-1.338750,...,0,51.204503,16.180513,98.613062,942.949585,15745.0,124.0,101762.211892,6385774.0,45643207.0
1,1,2016-01-02 00:00:00+00:00,DsLegAlRzecz,54.70,-9.532500,22.896110,102.65439,59.958332,1009.82965,-10.611667,...,0,51.204503,16.180513,98.613062,942.949585,15745.0,124.0,101762.211892,6385774.0,45643207.0
2,2,2016-01-03 00:00:00+00:00,DsLegAlRzecz,69.00,-12.282500,18.709612,107.50388,49.416668,1003.64520,-15.309583,...,0,51.204503,16.180513,98.613062,942.949585,15745.0,124.0,101762.211892,6385774.0,45643207.0
3,3,2016-01-04 00:00:00+00:00,DsLegAlRzecz,80.10,-10.632501,20.696087,118.76107,92.458336,985.55080,-12.167916,...,0,51.204503,16.180513,98.613062,942.949585,15745.0,124.0,101762.211892,6385774.0,45643207.0
4,4,2016-01-05 00:00:00+00:00,DsLegAlRzecz,100.50,-7.082500,13.324863,86.64323,95.750000,985.00410,-8.424168,...,0,51.204503,16.180513,98.613062,942.949585,15745.0,124.0,101762.211892,6385774.0,45643207.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66912,66912,2018-12-28 00:00:00+00:00,ZpSzczec1Maj,4.90,4.139000,24.535526,298.53427,100.000000,1003.10750,4.574417,...,60,53.712114,16.692517,23.818303,544.823893,8120.0,138.0,43274.445438,2695519.0,19724737.0
66913,66913,2018-12-29 00:00:00+00:00,ZpSzczec1Maj,8.16,3.889000,17.555307,256.10280,99.958336,1004.09380,4.422333,...,60,53.712114,16.692517,23.818303,544.823893,8120.0,138.0,43274.445438,2695519.0,19724737.0
66914,66914,2018-12-30 00:00:00+00:00,ZpSzczec1Maj,4.35,1.689000,27.002400,298.98737,95.958336,1000.37670,3.041083,...,60,53.712114,16.692517,23.818303,544.823893,8120.0,138.0,43274.445438,2695519.0,19724737.0
66915,66915,2018-12-31 00:00:00+00:00,ZpSzczec1Maj,7.44,1.839000,20.883102,271.19727,98.791664,1009.56620,1.770250,...,60,53.712114,16.692517,23.818303,544.823893,8120.0,138.0,43274.445438,2695519.0,19724737.0
